<div class='alert alert-info'>
<h1><center>Capstone Project</center></h1>
</div>

In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Creating a soup object

In [2]:
can_postal_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(can_postal_url).text

soup  = BeautifulSoup(source, 'xml')

In [3]:
import gc
gc.collect()

15

# Creating the dataframe from extracting the data from wikipedia link

In [4]:
table_contents=[]

table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
df = df.dropna()
df = df[df['Borough'] != 'Not assigned']

not_assigned_ids = df['Neighborhood'] == 'Not assigned'
df['Neighborhood'][not_assigned_ids] = df['Borough'][not_assigned_ids]

# Replace '/' by ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

# Merging the cells having the same postal code

In [6]:
temp = df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp = temp.reset_index(drop=False)
temp.rename(columns={'Neighborhood':'Neighborhood_joined'}, inplace=True)
temp.head()

PostalCode                     Neighborhood_joined
0        M1B                          Malvern, Rouge
1        M1C  Rouge Hill, Port Union, Highland Creek
2        M1E       Guildwood, Morningside, West Hill
3        M1G                                  Woburn
4        M1H                               Cedarbrae

In [7]:
df_final = pd.merge(df, temp, on='PostalCode')
del temp
del df
df_final.drop(['Neighborhood'], axis=1, inplace=True)
df_final.drop_duplicates(inplace=True)
df_final.rename(columns={'Neighborhood_joined':'Neighborhood'}, inplace=True)
df_final.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [8]:
df_final.shape

(103, 3)

In [9]:
#Loading the geo data as the usage of the geocoder timedout 

In [10]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')
geo_data.shape

(103, 3)

In [11]:
geo_data.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [12]:
#Merging the data 
geo_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_final=df_final.merge(geo_data,on='PostalCode',how='left')
del geo_data
df_final.head()

PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

In [13]:
df_final['Borough'].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'East York/East Toronto', 'Central Toronto', 'Mississauga',
       'Downtown Toronto Stn A', 'Etobicoke Northwest',
       'East Toronto Business'], dtype=object)

# Extracting the borough with Toronto

In [14]:
toronto_df = df_final[df_final['Borough'].str.contains("Toronto")]
del df_final
toronto_df.reset_index(drop=True, inplace=True)

toronto_df.head()

PostalCode           Borough               Neighborhood   Latitude  \
0        M5A  Downtown Toronto  Regent Park, Harbourfront  43.654260   
1        M5B  Downtown Toronto   Garden District, Ryerson  43.657162   
2        M5C  Downtown Toronto             St. James Town  43.651494   
3        M4E      East Toronto                The Beaches  43.676357   
4        M5E  Downtown Toronto                Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306

# Creating the toronto cluster visualization

In [17]:

# Create Toronto map
map_toronto = folium.Map(width=300,height=300,location=[43.65, -79.38], zoom_start=10)

# Add markers
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [23]:
#!pip install ipynbcompress
import os
from ipynbcompress import compress
filename = '/admin/Desktop/Toronto cluster.ipynb'
out = 'notebooks/Desktop/Toronto_cluster_final.ipynb'
# original size
os.stat(filename).st_size
# return bytes saved
compress(filename, output_filename=out, img_width=800, img_format='jpeg')
compress(filename, output_filename=out, img_width=800, img_format='png')
# defaults to img_width = 2048px and png compression
compress(filename, output_filename=out)
# overwrite existing notebook
compress(filename)


FileNotFoundError: [Errno 2] No such file or directory: '/admin/Desktop/Toronto cluster.ipynb'